In [17]:
import matplotlib.pyplot as plt
import numpy as np
import csv 
import pandas as pd
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier


percentualTreinamento = 0.7


dataset = {}
# Define the numeric labels to filter
labels={'M','B'}
labels_list = list(labels)


# Define column names based on dataset documentation
columns = [
    "ID", "Diagnosis",
    "Radius_mean", "Texture_mean", "Perimeter_mean", "Area_mean", "Smoothness_mean",
    "Compactness_mean", "Concavity_mean", "Concave_points_mean", "Symmetry_mean", "Fractal_dimension_mean",
    "Radius_se", "Texture_se", "Perimeter_se", "Area_se", "Smoothness_se",
    "Compactness_se", "Concavity_se", "Concave_points_se", "Symmetry_se", "Fractal_dimension_se",
    "Radius_worst", "Texture_worst", "Perimeter_worst", "Area_worst", "Smoothness_worst",
    "Compactness_worst", "Concavity_worst", "Concave_points_worst", "Symmetry_worst", "Fractal_dimension_worst"
]

# Load the dataset
dadosBrutos = pd.read_csv("../dataset/wdbc.data", names=columns, header=None)

# Drop the ID column (not useful for analysis)
dadosBrutos.drop(columns=["ID"], inplace=True)


# Filter rows where a column equals one of the labels
dadosBrutosLabel0 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[0]] 
dadosBrutosLabel1 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[1]] 

# metadata 
#print(dadosBrutos.head())



In [18]:
# Total number of samples

nAmostras_treinamento0 = len(dadosBrutosLabel0)
tamanhoTreinamento0 = int(nAmostras_treinamento0 * percentualTreinamento)

# Randomly select indices for group 1 - treinamento
indices_label0_treinamento = np.random.choice(dadosBrutosLabel0.index, size=tamanhoTreinamento0, replace=False)
# Select remaining indices for group 1 - teste
indices_label0_teste = dadosBrutosLabel0.index.difference(indices_label0_treinamento)

# Determine the size of group 2
nAmostras_treinamento1 = len(dadosBrutosLabel1)
tamanhoTreinamento1 = int(nAmostras_treinamento1 * percentualTreinamento)

# Randomly select indices for group 2 - treinamento
indices_label1_treinamento = np.random.choice(dadosBrutosLabel1.index, size=tamanhoTreinamento1, replace=False)
indices_label1_teste = dadosBrutosLabel0.index.difference(indices_label1_treinamento)

trainData = dadosBrutosLabel0.loc[indices_label0_treinamento]
buffer_trainData = dadosBrutosLabel1.loc[indices_label1_treinamento]
testData = dadosBrutosLabel0.loc[indices_label0_teste]
buffer_testData = dadosBrutosLabel1.loc[indices_label1_treinamento]

filtered_rows_Train = pd.concat([trainData,buffer_trainData], axis=0)
filtered_rows_Test = pd.concat([testData,buffer_testData], axis=0)

trainData = filtered_rows_Train.iloc[:, 1:31].to_numpy()
trainLabels= filtered_rows_Train.iloc[:, 0]
testData = filtered_rows_Test.iloc[:, 1:31].to_numpy()
testLabels= filtered_rows_Test.iloc[:, 0]

# aleatoriza ordem de treinamento
#indicesShuffledTranData = np.random.choice(trainData.index, size=1, replace=False)
#trainData = trainData.loc[indicesShuffledTranData]
#trainLabels = trainLabels.loc[indicesShuffledTranData]


trainLabels = trainLabels.replace({labels_list[0]: 0, labels_list[1]: 1}).to_numpy()
testLabels = testLabels.replace({labels_list[0]: 0, labels_list[1]: 1}) .to_numpy()

print(trainLabels)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [19]:
# Função para simular erro com médicos
def add_noise_to_labels(y_train, noise_level, noise_type):
    noisy_labels = y_train.copy()
    
    if noise_type == 'positive':
        # Médico 1: sempre positivo (rótulo 1)
        noise_indices = np.random.rand(len(y_train)) < noise_level
        noisy_labels[noise_indices] = 1  # Atribui 1 para todos os índices escolhidos
    elif noise_type == 'negative':
        # Médico 2: sempre negativo (rótulo 0)
        noise_indices = np.random.rand(len(y_train)) < noise_level
        noisy_labels[noise_indices] = 0  # Atribui 0 para todos os índices escolhidos
    
    return noisy_labels

## Loop Principal

In [20]:
import statistics
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


percentualErrosMaximo=0.05
stepErros=0.006

vector = np.arange(0, percentualErrosMaximo, stepErros)
vector = np.append(vector, [1.0])  # 1.0 para representar o cenário de médico extremo (sempre positivo)
vector = np.append(vector, [2.0])  # 2.0 para representar o cenário de médico extremo (sempre negativo)
print("Blocos de erro entrada: ", vector)


classifiers = {
    # "Random Forest": RandomForestClassifier(n_estimators = 15), # n_estimators =100
    # "SVM (RBF)": SVC(C=100,kernel='rbf', gamma=0.001,class_weight='balanced'),
    # "XGBoost": XGBClassifier(n_estimators = 20, objective = 'binary:logistic', tree_method = 'hist', eta = 0.1, # n_estimators = 100
    #                          max_depth = 3, enable_categorical = True),
    # "MLP": MLPClassifier(hidden_layer_sizes=(50), activation='tanh', solver='sgd',
    #                 max_iter=500, alpha=0.001)
    "Random Forest": RandomForestClassifier(n_estimators=50, max_depth=5, class_weight="balanced"),
    "SVM (RBF)": SVC(C=10, kernel='rbf', gamma=0.01, class_weight='balanced', probability=True),
    "XGBoost": xgb.XGBClassifier(n_estimators=50, max_depth=4, learning_rate=0.1, tree_method='hist', use_label_encoder=False, eval_metric='logloss'),
    "MLP": MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=500, alpha=0.001)
}
# MLPClassifier(hidden_layer_sizes=(50), activation='tanh', solver='lbfgs',
                    #max_iter=3000, alpha=0.001)

nRepeticoes=10
resultadosConsolidados_treino = []
resultadosConsolidados_teste = []
blocos = []
all_results=[]
resultadosAcc=[]


for nivelRuidoAtual in tqdm(vector):
    for indiceRepeticao in range(nRepeticoes):
        # Total number of samples

        dadosBrutosLabel0 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[0]] 
        dadosBrutosLabel1 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[1]] 

        nAmostras_treinamento0 = len(dadosBrutosLabel0)
        tamanhoTreinamento0 = int(nAmostras_treinamento0 * percentualTreinamento)
        tamanhoErroTreinamento0 = int(nAmostras_treinamento0 * (nivelRuidoAtual/2) )

        # Randomly select indices for group 1 - treinamento
        indices_label0_treinamento = np.random.choice(dadosBrutosLabel0.index, size=tamanhoTreinamento0, replace=False)
        indices_label0_erro =np.random.choice(indices_label0_treinamento, size=tamanhoErroTreinamento0, replace=False)

        # Select remaining indices for group 1 - teste
        indices_label0_teste = dadosBrutosLabel0.index.difference(indices_label0_treinamento)

        # Determine the size of group 2
        nAmostras_treinamento1 = len(dadosBrutosLabel1)
        tamanhoTreinamento1 = int(nAmostras_treinamento1 * percentualTreinamento)
        tamanhoErroTreinamento1 = int(nAmostras_treinamento0 * (nivelRuidoAtual/2) )

        # Randomly select indices for group 2 - treinamento
        indices_label1_treinamento = np.random.choice(dadosBrutosLabel1.index, size=tamanhoTreinamento1, replace=False)
        indices_label1_teste = dadosBrutosLabel0.index.difference(indices_label1_treinamento)
        indices_label1_erro =np.random.choice(indices_label1_treinamento, size=tamanhoErroTreinamento1, replace=False)

        # Introduz erro
        dadosLabel0=dadosBrutosLabel0
        dadosLabel1=dadosBrutosLabel1

        if nivelRuidoAtual == 1.0:
            # Médico 1: sempre diagnóstico positivo (1) para o Label0
            dadosLabel0["Diagnosis"] = add_noise_to_labels(dadosLabel0["Diagnosis"], nivelRuidoAtual, "positive")
        elif nivelRuidoAtual == 2.0:
            # Médico 2: sempre diagnóstico negativo (0) para o Label1
            dadosLabel1["Diagnosis"] = add_noise_to_labels(dadosLabel1["Diagnosis"], nivelRuidoAtual, "negative")
        else:
            # Introduzindo erro com base no nível de ruído normal
            dadosLabel0.loc[indices_label0_erro, "Diagnosis"] = dadosLabel0.loc[indices_label0_erro, "Diagnosis"].map({"M": "B", "B": "M"})
            dadosLabel1.loc[indices_label1_erro, "Diagnosis"] = dadosLabel1.loc[indices_label1_erro, "Diagnosis"].map({"M": "B", "B": "M"})

        trainData = dadosLabel0.loc[indices_label0_treinamento]
        buffer_trainData = dadosLabel1.loc[indices_label1_treinamento]
        testData = dadosLabel0.loc[indices_label0_teste]
        buffer_testData = dadosLabel1.loc[indices_label1_treinamento]

        filtered_rows_Train = pd.concat([trainData,buffer_trainData], axis=0)
        filtered_rows_Test = pd.concat([testData,buffer_testData], axis=0)

        #aleatoriza ordem de treinamento
        indicesShuffledTranData = np.random.choice(filtered_rows_Train.index, size=1, replace=False)
        trainData = filtered_rows_Train.loc[indicesShuffledTranData]
        trainLabels = filtered_rows_Train.loc[indicesShuffledTranData]


        trainData = filtered_rows_Train.iloc[:, 1:31]
        trainLabels= filtered_rows_Train.iloc[:, 0]
        testData = filtered_rows_Test.iloc[:, 1:31]
        testLabels= filtered_rows_Test.iloc[:, 0]


        trainLabels = filtered_rows_Train.loc[:, "Diagnosis"].map({"M": 0, "B": 1}).to_numpy()
        testLabels = filtered_rows_Test.loc[:, "Diagnosis"].map({"M": 0, "B": 1}).to_numpy()
          
        # Calcula o nivel de ruido real baseado no número de indices flipados divididio pelo numero total de indices 
        nivelRuidoAtual_real=(len(indices_label0_erro)+len(indices_label1_erro))/len(trainLabels)
        
        ### Treinamentos
        for name, model in classifiers.items():
            model.fit(trainData, trainLabels)
            resultadoTreinamento=model.predict(trainData)
            resultadoTeste=model.predict(testData)
            acc_teste = ( sum((testLabels==resultadoTeste))/len(testLabels) )
            resultadosAcc.append([nivelRuidoAtual_real, name, indiceRepeticao,acc_teste])
            for true_label, pred_label in zip(resultadoTeste, testLabels):
                all_results.append([nivelRuidoAtual_real, name, indiceRepeticao, true_label, pred_label, acc_teste])

        
      
    # print("observados_treinamento=",observados_treinamento)
    # print("soma=",sum(trainLabels==observados_treinamento))æ
    # print("len=",len(trainLabels))

    # print(sum(trainLabels==observados_treinamento)/len(trainLabels))

print(blocos)

df_results = pd.DataFrame(all_results, columns=["Noise Level", "Classifier", "IndexRep", "True Label", "Predicted Label", "Accuracy"])
resultadosExp = pd.DataFrame(resultadosAcc, columns=["Noise Level", "Classifier", "IndexRep", "Accuracy"])


Blocos de erro entrada:  [0.    0.006 0.012 0.018 0.024 0.03  0.036 0.042 0.048 1.    2.   ]


/home/ana/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [19:30:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ana/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [19:30:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ana/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [19:30:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ana/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [19:30:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/ana/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [19:30:07] WARNING: /workspace/src/learner.cc:740: 
P

ValueError: Cannot take a larger sample than population when 'replace=False'

In [21]:
import os
print(resultadosExp)
string = "./resultados/estudo_piloto.csv" 
if not os.path.exists(string):
    resultadosExp.to_csv(string, index=True)

     Noise Level     Classifier  IndexRep  Accuracy
0       0.000000  Random Forest         0  0.992188
1       0.000000      SVM (RBF)         0  1.000000
2       0.000000        XGBoost         0  0.996094
3       0.000000            MLP         0  0.929688
4       0.000000  Random Forest         1  0.976562
..           ...            ...       ...       ...
355     0.040302            MLP         8  0.914062
356     0.040302  Random Forest         9  0.960938
357     0.040302      SVM (RBF)         9  0.992188
358     0.040302        XGBoost         9  0.968750
359     0.040302            MLP         9  0.820312

[360 rows x 4 columns]
